<a href="https://colab.research.google.com/github/nazike/dataoptions/blob/main/OptionsAnalysisPricing191021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#read file from the github repo
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
df = pd.read_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/DataIn/V2.csv')
#merge two dataframes
df.columns = columns

In [3]:
df.describe()

,asset_price,maturity,rate,div,ivol,eu_price,am_price
count,100455.000000,100455.000000,1.004550e+05,1.004550e+05,100455.000000,100455.000000,100455.000000
mean,98.141144,1.214371,3.479940e-02,3.525762e-02,0.339526,15.501890,15.961782
std,16.859969,1.256565,2.450852e-02,2.471843e-02,0.156516,10.418466,10.663400
min,70.000660,0.100010,5.427117e-07,5.427117e-07,0.000015,0.000000,0.500121
25%,83.673626,0.423637,1.555361e-02,1.585122e-02,0.213415,6.687584,6.905906
50%,97.373486,0.725755,3.107879e-02,3.154336e-02,0.345592,14.226921,14.696291
75%,112.142653,1.246729,4.666650e-02,4.707699e-02,0.473691,23.108803,23.921224
max,129.999578,4.999891,9.999842e-02,9.999347e-02,0.599988,63.146201,63.146473


In [6]:
# Read the data

X_full = df[(df['ivol']>0.12)  (df['maturity']<0.5)].copy()
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
X_full.columns = columns
X_full['target'] = ((X_full['am_price']-X_full['eu_price'])/X_full['am_price']).copy()

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['target'], inplace=True)
y = X_full.target
y_amer = X_full['am_price']
X_full.drop(['am_price'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/DataOut/v22fulldata.csv', index=False)
X_full.drop(['target'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/DataOut/v22fulldatatrain.csv', index=False)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality 
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
#model = XGBRegressor(n_estimators=10000, random_state=0, learning_rate=0.05, n_jobs=4)

model = XGBRegressor(max_depth=6, reg_alpha=0.0002433756223211188, colsample_bytree=0.8242913620513932, learning_rate=0.05640249301683806, gamma=4.5885092732685184e-05,
                        reg_lambda=0.014400178832201075, min_child_weight=0.003963394614607396, 
                       n_estimators=10000, n_jobs=4, random_state=0, subsample=0.8586706877598019)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['asset_price', 'maturity',
                                                   'rate', 'div', 'ivol',
                                                   'eu_price']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtre...
                            

In [7]:
# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MSE',mean_squared_error(y_valid, preds))

print('MAE:', mean_absolute_error(y_valid, preds))

MSE 1.5180348030382164e-05
MAE: 0.0016669033852988306


In [8]:
preds_full=clf.predict(X_full)
preds_full

array([ 0.00031262,  0.0054054 ,  0.01378856, ...,  0.11391692,
       -0.00071057,  0.00125649], dtype=float32)

In [10]:
#create a dataframe with the predicted values and write it to csv   
predicted_df=df[df['ivol']>0.12].copy()
predicted_df['predicted_target']=preds_full
predicted_df['predicted_am_price']=predicted_df['predicted_target']*predicted_df['eu_price']+predicted_df['eu_price']
predicted_df.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/DataOut/OPAv22sigma_predicted.csv', index=False)

In [14]:
#calculation of the neural net time 
X_sample = X_full.iloc[0:1].copy()


import timeit
def test_nn():
    return clf.predict(X_sample)

loop = 1000

result_nn = timeit.timeit('test_nn()', globals=globals(), number=loop)
print(result_nn, 'seconds per loop times of test function')
print(result_nn/loop, 'seconds per each clf.predict()')

1.869605125000021 seconds per loop times of test function
0.001869605125000021 seconds per each clf.predict()


In [15]:
#function to calculate binomial option price

import numpy as np

def binomial_put(S, K, T, R, div, sig, n):
    
    h = T/n
    Rinv = np.exp(-R*h)
    a = np.exp((R-div)*h)
    b2 = (a**2)*(np.exp(h*sig**2)-1)
    tmp = a**2 + b2 + 1
    u = (tmp + (tmp**2 - 4*a**2)**0.5)/(2*a)
    d = 1/u
    p = (a-d)/(u-d)
    q = 1-p
    pp = Rinv*p
    qq = Rinv*q
    x = 2*n + 1
    
    s = np.zeros(x)
    s[n] = S
    
    for j in range(1,n+1):
        s[n+j] = s[n-1+j]*u
        s[n-j] = s[n+1-j]*d
        
    v = np.zeros(x)
    
    for j in range(1,x+2,2):
        v[j-1] = max(K-s[j-1],0)
        
    for i in range(n-1,0,-1):
        for j in range(-i,i+2,2):
            j1 = j+n+1
            v[j1-1] = max(K-s[j1-1],pp*v[j1]+qq*v[j1-2])
    
    y=pp*v[n+1]+qq*v[n-1]
    
    return y

In [16]:
#parameters for the binomial put option price calcualtion , similar to X_full.iloc[0:1]

S = 91.063192
K = 100.0
T = 0.64276
R = 0.073981	
div = 0.069325	
sig = 0.312864
n=250


result = binomial_put(S, K, T, R, div, sig, n)


In [17]:
#binomial option price calcualtion time
def test_bt():
    return binomial_put(S, K, T, R, div, sig, n)

loop = 1000

result_bt = timeit.timeit('test_bt()', globals=globals(), number=loop)
print(result_bt, 'seconds per loop times of test function')
print(result_bt/loop, 'seconds per each binomial_put')

21.657898457999977 seconds per loop times of test function
0.021657898457999977 seconds per each binomial_put


In [18]:
#result, which shows how much faster the neural net is than the binomial tree
print(result_bt/result_nn, 'times nn faster than bt')

11.584210039004752 times nn faster than bt
